In [1]:
import numpy as np
import tensorflow as tf
import tools
import metrics
import keras_losses as kl
import keras_models as kms
import bnn_predictions as bnn

/home/astrophysics/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
run_ns = range(1,11)
#architecture parameters
num_inputs = 14
num_outputs = 2
layer_sizes = []
m = 64 #total number of records
training_size = 48. / 64.
test_size = 1 - training_size
test_num = int(np.ceil(m * test_size))

In [3]:
#following loops could easily be included into same loop, but currently have them separate for clarity

In [4]:
MPE_test_losses = []
MPE_test_accuracies = []
for run_n in run_ns:
    #load data
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(run_n) + ".csv", delimiter = ',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(run_n) + ".csv", delimiter = ',')
    #load keras MPE model
    model_r = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/kaggle/FIFA_2018_Statistics_slp_sm_" + str(run_n) + ".h5", custom_objects={'sum_of_categorical_crossentropy': kl.sum_of_categorical_crossentropy})
    #calculate evaluation metrics for keras MPE
    y_pred_r = model_r.predict(x_test)
    y_test, y_pred_r = tools.check_dtypes(y_test, y_pred_r)
    test_loss_r = metrics.categorical_crossentropy(y_test, y_pred_r) #averaged over test set size
    test_acc_r = metrics.categorical_accuracy(y_test, y_pred_r)
    MPE_test_losses.append(test_loss_r)
    MPE_test_accuracies.append(test_acc_r)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
#load in model from first run
x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(1) + ".csv", delimiter = ',')
y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(1) + ".csv", delimiter = ',')
#load keras MPE model
model_r = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/kaggle/FIFA_2018_Statistics_slp_sm_" + str(1) + ".h5", custom_objects={'sum_of_categorical_crossentropy': kl.sum_of_categorical_crossentropy})
#calculate evaluation metrics for keras MPE
y_pred_r = model_r.predict(x_test)

In [14]:
y_pred_r

array([[0.19832076, 0.80167925],
       [0.28487757, 0.71512246],
       [0.12846611, 0.87153393],
       [0.4309077 , 0.56909233],
       [0.51794267, 0.48205733],
       [0.71066123, 0.28933877],
       [0.2642904 , 0.7357096 ],
       [0.44528648, 0.55471355],
       [0.67772263, 0.32227734],
       [0.40895075, 0.59104925],
       [0.5386699 , 0.4613301 ],
       [0.24792802, 0.752072  ],
       [0.565747  , 0.434253  ],
       [0.21927941, 0.78072053],
       [0.20592867, 0.7940714 ],
       [0.90201044, 0.09798948]], dtype=float32)

In [6]:
#this is for testing purposes 
#model.evaluate should be used with caution, as if mode contains regularisation, the method will include it
score_r = model_r.evaluate(x_test, y_test, verbose=0, batch_size=test_num)
print('Test loss:', score_r[0])
print('Test accuracy:', score_r[1])

('Test loss:', 11.766074180603027)
('Test accuracy:', 0.75)


In [6]:
print tools.get_km_weight_magnitudes(model_r)

[[-0.6761643   0.6053163 ]
 [-0.06116175  0.54149187]
 [ 0.07489242 -0.07099907]
 [-0.15657759  0.23196256]
 [-0.30747503  0.8516517 ]
 [ 0.2835439  -0.25466424]
 [-0.4790744   0.46951705]
 [-0.06768923  0.06765081]
 [ 0.06722418 -0.02446986]
 [-0.7356136   0.7141812 ]
 [ 0.10059257 -0.07851817]
 [-0.01864189 -0.00429505]
 [-0.00399538  0.00165122]
 [ 0.09430664 -0.18412459]]
[ 0.03261434 -0.03261434]
1.9363669
0.046123635
1.9369162
([1.9363669, 0.046123635], 1.9369161)


In [7]:
#cpp bnn

In [16]:
# bnn expected estimates
bnn_test_losses = []
bnn_test_accuracies = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/FIFA_2018_Statistics_slp_sm_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(run_n) + ".csv", delimiter=',')
    bmodel = kms.slp_sm(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.categorical_crossentropy(y_test, by_pred)
    b_test_acc = metrics.categorical_accuracy(y_test, by_pred)
    bnn_test_losses.append(b_test_loss)
    bnn_test_accuracies.append(b_test_acc)

In [9]:
# bnn MPE estimates
bnn_MPE_test_losses = []
bnn_MPE_test_accuracies = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/FIFA_2018_Statistics_slp_sm_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(run_n) + ".csv", delimiter=',')
    bmodel = kms.slp_sm(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.MPE_prediction()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.categorical_crossentropy(y_test, by_pred)
    b_test_acc = metrics.categorical_accuracy(y_test, by_pred)
    bnn_MPE_test_losses.append(b_test_loss)
    bnn_MPE_test_accuracies.append(b_test_acc)

In [12]:
print tools.get_km_weight_magnitudes(bnn_model.model)

[[-0.85722125  2.201045  ]
 [-1.128426   -0.9618036 ]
 [-0.01186933 -0.07161662]
 [-0.19148481  1.4325533 ]
 [-1.3325505  -1.288243  ]
 [ 1.5406443   0.49094737]
 [ 1.111408    2.7693534 ]
 [ 0.07202915  0.6641578 ]
 [-0.30221823  1.0426121 ]
 [-0.5718678   0.97703904]
 [ 1.7841057  -0.41787377]
 [-0.16570689 -0.02360797]
 [-1.1184111  -1.1554649 ]
 [-0.08572011 -0.22775735]]
[-0.9668519 -1.263194 ]
5.810641
1.5907425
6.024451
([5.810641, 1.5907425], 6.024451)


In [8]:
print MPE_test_losses
print np.mean(MPE_test_losses)

[0.646150223639557, 0.5942609350490429, 0.32318632259460517, 0.546431633332441, 0.29450505214739575, 0.287850269842976, 0.2842792626669594, 0.3321624040501262, 0.30422067663949814, 0.32166057952266724]
0.3934707359485269


In [17]:
print bnn_test_losses
print bnn_test_accuracies

[0.366062825470115, 0.6284091941729878, 0.24924489948015272, 0.5710904364299889, 0.24501502361292296, 0.3579497096714941, 0.27207251425240053, 0.3570041982039275, 0.34970751339715545, 0.35664656718022647]
[0.75, 0.625, 0.9375, 0.625, 0.9375, 0.8125, 0.875, 0.8125, 0.8125, 0.8125]


In [14]:
print bnn_MPE_test_losses
print bnn_MPE_test_accuracies

[0.8409075339178537, 0.8671017332912981, 0.550785078166951, 0.7411403889394357, 0.25798395642442773, 0.7648644009996977, 0.27389507183596595, 0.6367651840596237, 0.9569289182702858, 0.5288760076862427]
[0.5625, 0.625, 0.75, 0.625, 0.9375, 0.8125, 0.875, 0.8125, 0.625, 0.75]


In [5]:
#just look at first run from cpp chains
chains_file = '../cpp_models/cpp_chains/FIFA_2018_Statistics_slp_sm_' + str(1) + '.txt'
x_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_x_te_" + str(1) + ".csv", delimiter=',')
y_test = np.genfromtxt("../../data/kaggle/FIFA_2018_Statistics_y_te_" + str(1) + ".csv", delimiter=',')
bmodel = kms.slp_sm(num_inputs, num_outputs, layer_sizes)
bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
bnn_model.setup_LL('categorical_crossentropy')

In [9]:
bnn_model.y_pred_paramnames('FIFA_2018_Statistics_slp_sm_pred_' + str(1))

In [11]:
bnn_model.y_pred_chains('keras_chains/FIFA_2018_Statistics_slp_sm_pred_' + str(1) + '_' + str(1), test_index = 1)

array([[3.18120668e-154, 7.60750497e+002, 3.63046212e-007,
        9.99999642e-001],
       [7.49849493e-149, 7.36007763e+002, 3.97993386e-001,
        6.02006614e-001],
       [5.00839506e-140, 6.95366406e+002, 9.41447735e-001,
        5.85522838e-002],
       ...,
       [1.56641763e-002, 1.61902841e+001, 9.68318820e-001,
        3.16811278e-002],
       [1.71406913e-002, 1.60101263e+001, 9.20546710e-001,
        7.94532746e-002],
       [2.66659221e-002, 1.51262639e+001, 9.51721430e-001,
        4.82785702e-002]])

In [10]:
bnn_model.MPE_prediction()

array([[6.9351736e-05, 9.9993062e-01],
       [7.4244219e-01, 2.5755775e-01],
       [4.2735225e-01, 5.7264781e-01],
       [9.2278069e-01, 7.7219315e-02],
       [2.0204382e-01, 7.9795611e-01],
       [7.2129518e-01, 2.7870479e-01],
       [5.0906497e-01, 4.9093500e-01],
       [4.5925999e-04, 9.9954081e-01],
       [3.8758034e-01, 6.1241972e-01],
       [1.1770270e-01, 8.8229728e-01],
       [9.8896092e-01, 1.1039097e-02],
       [2.9892327e-02, 9.7010761e-01],
       [7.9502410e-01, 2.0497583e-01],
       [9.3773438e-04, 9.9906224e-01],
       [5.5223264e-02, 9.4477671e-01],
       [9.2320722e-01, 7.6792717e-02]], dtype=float32)

In [12]:
bnn_model.MLE_prediction()

array([[1.7810422e-05, 9.9998224e-01],
       [9.5172143e-01, 4.8278570e-02],
       [2.7556656e-02, 9.7244328e-01],
       [9.9632108e-01, 3.6789160e-03],
       [5.9966612e-01, 4.0033394e-01],
       [8.6174303e-01, 1.3825704e-01],
       [3.3111984e-01, 6.6888016e-01],
       [3.4005450e-05, 9.9996603e-01],
       [9.9820960e-01, 1.7903876e-03],
       [4.8865047e-01, 5.1134950e-01],
       [9.9952435e-01, 4.7566410e-04],
       [2.6356284e-02, 9.7364378e-01],
       [8.0469024e-01, 1.9530970e-01],
       [9.4777233e-06, 9.9999046e-01],
       [2.3397352e-01, 7.6602656e-01],
       [8.8386852e-01, 1.1613145e-01]], dtype=float32)